In [ ]:
import pandas as pd
import numpy as np
import random

# Impostiamo il "seed" del generatore casuale. Ogni volte che lo script verrà eseguito i numeri casuali generati saranno gli stessi
np.random.seed(42)

# --- DATI ---
# Verrà creato un dizionario dove le chiavi saranno i nomi delle colonne, e ogni valore sarà la lista/serie di dati di quella colonna

data_prodotto = {
    'product_id': range (1,11), # valori da 1 a 10
    'category': np.random.choice(["Electronics", "Home", "Fashion"], 10), # sceglie a caso una categoria tra quelle indicate, generando 10 valori
    'brand': np.random.choice(["BrandA", "BrandB", "BrandC"], 10), # sceglie a caso uno dei 3 brand, generando sempre 10 valori
    'price': np.random.uniform(20,500,10).round(2) # genera 10 numeri casuali in virgola mobile tra 20 e 500 e poi arrotonda i numeri a 2 decimali
}


# Generiamo DATE e VENDITE relative all'anno 2023
dates = pd.date_range(start='2023-01-01', end = '2023-12-31', freq='D') # creaiamo un array con una sequenza di date giornaliere, "D" = daily, dal 1 gennaio al 31 dicembre 2023 inclusi
data_vendite = [] # creata una lista vuota, nella quale verranno aggiunte tutte le "righe" delle vendite, una per ogni vendita
for date in dates: # per ogni giorno contenuto in dates, esegue il blocco dentro il for
  # Generiamo Tra 0 e 5 vendite al giorno
  n_sales = np.random.randint(0,6) #decide quante vendite ci saranno per ogni giorno, tra 0 e 5, quindi possono esserci anche 0 vendite
  for _ in range(n_sales):  # esegue un ciclo n_sales volte
    data_vendite.append({  # aggiunge alla lista data_vendite un dizionario che rapprenseta un singola riga di vendita
        'date': date,
        'product_id': np.random.randint(1,11), # sceglie un prodotto casuale tra 1 e 10
        'quantity': np.random.randint(1,4), # sceglie la quantità venduta tra 1 e 3
        'customer_id': np.random.randint(100,200) # assegna un id cliente casuale tra 100 e 199
    })

df_products = pd.DataFrame(data_prodotto) # creaiamo un DataFrame a partire dal dizionario data_prodotto, come risultato avremo una tabella con 10 righe e 4 colonne
df_sales = pd.DataFrame(data_vendite) # converte la lista di dizioonari data_vendite in un DataFrame. Ogni dizionario è una riga di vendite. il n di righe dipende da quante vendite casuali sono generate

print("Dataset iniziali creati")
print(df_products.head())
print(df_sales.head())





Dataset iniziali creati
   product_id     category   brand   price
0           1      Fashion  BrandC  107.28
1           2  Electronics  BrandC  108.03
2           3      Fashion  BrandA  166.04
3           4      Fashion  BrandC  271.88
4           5  Electronics  BrandB  227.33
        date  product_id  quantity  customer_id
0 2023-01-01           3         3          163
1 2023-01-01           9         3          150
2 2023-01-01           7         1          172
3 2023-01-01           7         2          103
4 2023-01-01           9         2          108


# Parte 1: Inquinamento dei Dati (Generazione di Null e Duplicati)**
I dati reali non sono mai perfetti. Dobbiamo, quindi, simulare in dataset "sporco".


1.   **Generazione di valori Nulli:**


*   Utilizzando `numpy` e l'indicizzazione di pandas, assegna `np.nan` (valori nulli) al 5% della colonna `quantity` in `df_sales`
*   Assegna `np.nan` alla colonna `brand` di `df_products` per i prodotti con `product_id` pari




2.   **Generazione di Duplicati**:


*   Crea una copia delle prime 10 righe di df_sales e concatenale al dataframe originale. Ora hai dei duplicati intenzionali





# Generazione di valori nulli

In [ ]:
# Calcoliamo il 5% delle righe, arrotondato per eccesso a minimo 1
n_nan = max(1,int(len(df_sales)*0.05))

# Scegliamo gli indici casuali di n_nan righe
idx_nan = np.random.choice(df_sales.index, size=n_nan, replace=False)

# Assegniamo i valori NaN alla colonna quantity delle righe con indice idx_nan
df_sales.loc[idx_nan, 'quantity'] = np.nan






In [ ]:
# L'idea è quella di creare una maschera booleana con % 2 == 0 (così troviamo i product_id pari)
# e usiamo .loc per inserire i valori NaN alla colonna brand di df_products

mask_pari = (df_products['product_id'] % 2 == 0)
df_products.loc[mask_pari, 'brand'] = np.nan



# Generazione dei duplicati

In [ ]:
# Facciamo la copia delle prime 10 righe
df_sales_copy10 = df_sales.head(10).copy()

# Adesso le concateniamo al DataFrame originale tramite il metodo concat
df_sales = pd.concat([df_sales, df_sales_copy10], ignore_index=True) # ignore_index non replica gli indici

# PARTE 2: PULIZIA E CONTROLLO DEI DATI
Ora che abbiamo i dati sporchi, bisogna procedere con la "pulizia dei dati"



*   Visualizza il conteggio dei valori nulli per ogni colonna di entrambi i DataFrame
*   Gestisci i null in `df_sales['quantity'] `: sostiuiscili con la media della quantità (arrotondata all'intero più vicino)


*   Gestisci i null in `df_products['brand']`: sostiuiscili con la stringa "Unknown"
*   Identifica e rimuovi le righe duplicate in df_sales mantenendo solo la prima occorenza.





In [ ]:
# Conteggio dei valori nulli per colonna nel DataFrame df_sales
nulli_sales = df_sales.isnull().sum()
print("Valori nulli nel DataFrame df_sales:")
print(nulli_sales)

# Conteggio dei valori nulli per colonna nel DataFrame df_products
nulli_products = df_products.isnull().sum()
print("Valori nulli nel DataFrame df_products:")
print(nulli_products)


Valori nulli nel DataFrame df_sales:
date            0
product_id      0
quantity       44
customer_id     0
dtype: int64
Valori nulli nel DataFrame df_products:
product_id    0
category      0
brand         5
price         0
dtype: int64


In [ ]:
# 1 - Calcoliamo la media della colonna ignorando i NaN (pandas lo fa di default) e arrotondandola all'intero più vicino
media_quantity = int(round(df_sales['quantity'].mean()))

# 2 - Sostituiamo i NaN con il valore della media calcolato, il metodo fillna(x), sostituisce tutti i NaN con il valore x per la colonna indicata
df_sales['quantity'] = df_sales['quantity'].fillna(media_quantity)

# Aggiungiamo una verifica
print("NaN rimasti in quantity:", df_sales['quantity'].isna().sum())
print("Valore usato per imputazione:", media_quantity)


NaN rimasti in quantity: 0
Valore usato per imputazione: 2


In [ ]:
# Sostituiamo i valori NaN presenti nella colonna brand di df_products con la stringa "Unknown"
df_products['brand'] = df_products['brand'].fillna("Unknown")

# Aggiungiamo una verifica
print("NaN rimasti in df_products['brand']:", df_products['brand'].isna().sum())

NaN rimasti in df_products['brand']: 0


In [ ]:
# Identifichiamo ed eliminiamo le righe duplicate  in df_sales e manteniamo la prima occorenza

# Calcoliamo intanto quante righe duplicate ci sono prima della rimozione
print("Duplicati prima:", df_sales.duplicated().sum())

# Rimuoviamo i duplicati mantenendo la prima occorenza
# Per farlo usiamo il metodo drop_duplicates(keep='first') che elimina tutte le copie successive e lascia la prima
# Con reset_index(drop=True), invece, ricreiamo un indice pulito del nuovo DataFrame che va da 0...a N-1
df_sales = df_sales.drop_duplicates(keep='first').reset_index(drop=True)

# Verifichiamo
print("Duplicati dopo:", df_sales.duplicated().sum())
print("Righe totali dopo:", len(df_sales))

Duplicati prima: 10
Duplicati dopo: 0
Righe totali dopo: 873


# PARTE 3: INTEGRAZIONE E GESTIONE DELLE DATE



1.   **Analisi Temporale**


*   Assicurati che la colonna `date` in `df_sales` sia in formato datetime
*   Crea due nuove colonne in `df_sales`: `year` e `month`. Estrai l'anno e il mese della data


*   Calcola una colonna `total_price` che sia il risultato di `quantity*price` (attenzione: il prezzo è in `df_products`, non in `df_sales`)




2.   **Merge (Unione)**


*   Esegui un `merge` (left join) tra `df_sales` e `df_products` sulla base di `product_id` per portare le informazioni sui prodotti (categoria,prezzo,marca) dentro il dataframe delle vendite





In [ ]:
# Assicura che df_sales['date'] sia in formato datetime
df_sales['date'] = pd.to_datetime(df_sales['date'], errors="coerce")

# pd.to_datetime() converte la colonna in datetime
# errors=coerce trasforma eventuali valori non convertibili in NaT (null datetime), invece di lanciare un errore

# Facciamo una verifica
print(df_sales['date'].dtype)
print("NaT in date:", df_sales['date'].isna().sum())

datetime64[ns]
NaT in date: 0


In [ ]:
# Creo le colonne year e month estraendole dalla data
df_sales['year'] = df_sales['date'].dt.year
df_sales['month'] = df_sales['date'].dt.month

# In questo modo ottengo il mese con il nome invece che con il numero
# df_sales['month_name'] = df_sales['date'].dt.strftime('%B')

In [ ]:
# Calcolo di una colonna che sia il risultato di qauntity*price
df_sales = df_sales.merge(df_products[['product_id', 'price']], on = 'product_id', how = 'left')
df_sales['total_price'] = df_sales['quantity'] * df_sales['price']

print(df_sales.columns.tolist()) # Controlliamo numero e nome delle colonne di df_sales

df_sales = df_sales.merge(
    df_products[['product_id', 'category', 'brand']],
    on='product_id',
    how='left'
)

print(df_sales.columns.to_list()) # controlliamo di nuovo dopo il merge





['date', 'product_id', 'quantity', 'customer_id', 'year', 'month', 'price', 'total_price']
['date', 'product_id', 'quantity', 'customer_id', 'year', 'month', 'price', 'total_price', 'category', 'brand']


# Parte 4: Analisi esplorativa (Groupby, Pivot, Filter)
Utilizzando il dataframe ottenuto nel punto precedente:


*   Filtra le vendite solo per la categoria Electronics
*   Di queste vendite mostra le top 3 transazioni per `total_price` più alto



In [ ]:
# Filtro le vendite solo per la categoria Electronics
df_sales_electronics = df_sales.loc[df_sales['category'] == 'Electronics'].copy()
print(df_sales_electronics)
print("Numero vendite Electronics:", len(df_sales_electronics))

          date  product_id  quantity  customer_id  year  month   price  \
5   2023-01-02           5       2.0          183  2023      1  227.33   
9   2023-01-04           6       2.0          103  2023      1  159.79   
10  2023-01-05           2       2.0          143  2023      1  108.03   
11  2023-01-05           2       2.0          161  2023      1  108.03   
15  2023-01-06           2       3.0          152  2023      1  108.03   
..         ...         ...       ...          ...   ...    ...     ...   
853 2023-12-26           2       2.0          197  2023     12  108.03   
855 2023-12-26           6       3.0          139  2023     12  159.79   
861 2023-12-28           6       2.0          126  2023     12  159.79   
864 2023-12-28           2       1.0          102  2023     12  108.03   
866 2023-12-29           6       3.0          151  2023     12  159.79   

     total_price     category    brand  
5         454.66  Electronics   BrandB  
9         319.58  Electronics

In [ ]:
# Mostro le prime 3 vendite per total_price più alto, ordinando la colonna in modo decrescente e prendendo i primi 3 risultati
top3 = df_sales_electronics.sort_values('total_price', ascending=False).head(3)

# Mostra le colonne più utili
print(top3[['date', 'product_id', 'category', 'brand', 'quantity', 'price', 'total_price']])


          date  product_id     category   brand  quantity   price  total_price
29  2023-01-12           5  Electronics  BrandB       3.0  227.33       681.99
116 2023-02-19           5  Electronics  BrandB       3.0  227.33       681.99
211 2023-03-30           5  Electronics  BrandB       3.0  227.33       681.99


**Grouping**


*   Raggruppa i dati per `category`. Calcola il totale delle vendite (`sum` di `total_price` e la quantità media venduta)
*   Raggruppa i dati `per month`. Quale mese ha generato il fatturato più alto?



In [ ]:
# Raggruppo i dati per category e calcolo il totale delle vendite e la quantità media venduta, faccio anche il conteggio del numero di vendite
group_category = (
    df_sales
    .groupby('category', dropna=False)
    .agg(
        n_transactions=('product_id', 'size'),
        total_revenue=('total_price', 'sum'),
        avg_quantity=('quantity', 'mean')
    )
    .reset_index()
)

print(group_category)


      category  n_transactions  total_revenue  avg_quantity
0  Electronics             255       89273.44      2.117647
1      Fashion             535      210721.54      1.953271
2         Home              83       14435.36      2.000000


In [ ]:
# Raggruppo i dati per month
group_month = (
    df_sales
    .groupby('month', dropna=False)
    .agg(
        month_fatturato=('total_price', 'sum'),
        n_vendite=('product_id', 'size'),
        media_quantity=('quantity', 'mean')
    )
    .reset_index()
    .sort_values('month_fatturato', ascending=False)
)

print(group_month)

    month  month_fatturato  n_vendite  media_quantity
9      10         33928.15         89        2.056180
8       9         29283.95         78        1.935897
5       6         29244.83         74        2.081081
6       7         28865.34         82        2.012195
11     12         28705.03         88        2.022727
2       3         26630.36         78        1.948718
3       4         26157.90         73        2.027397
1       2         25329.14         73        1.917808
0       1         24257.21         65        1.984615
4       5         22571.27         58        2.034483
7       8         20054.12         64        1.953125
10     11         19403.04         51        2.117647


In [ ]:
# Il mese con fatturato più alto
best_month = group_month.iloc[0]
print(f"Mese top: {int(best_month['month'])} con fatturato= {best_month['month_fatturato']:.2f}")


Mese top: 10 con fatturato= 33928.15


**Pivot table**


*   Crea una Pivot table che abbia come indice (`index`) la `category`, come colonne (`columns`) il `month`, e come valori (`values`) la somma delle quantità vendute. Fillate i valori mancanti con 0



In [ ]:
# Pivot table: index=category, columns=month, values=sum(quantity)
pivot_cat_month_qty = (
    df_sales.pivot_table(
        index='category',
        columns='month',
        values='quantity',
        aggfunc='sum',
        fill_value=0
    )
)

print(pivot_cat_month_qty)


month          1     2      3     4     5     6      7     8     9      10  \
category                                                                     
Electronics  28.0  46.0   42.0  45.0  38.0  50.0   40.0  36.0  62.0   58.0   
Fashion      89.0  87.0  100.0  84.0  63.0  85.0  102.0  72.0  85.0  118.0   
Home         12.0   7.0   10.0  19.0  17.0  19.0   23.0  17.0   4.0    7.0   

month          11     12  
category                  
Electronics  43.0   52.0  
Fashion      54.0  106.0  
Home         11.0   20.0  


# Parte 5: Trasformazione e Normalizzazione Dati

**Normalizzazione (Min-Max Scaling):**



*   Utilizzando solo `numpy` (non funzioni pronte di sklearn), calcolare la normalizzazione Min-Max della colonna `total_price`.
*   Formula: xnew​=max(x)−min(x)x−min(x)​


*   Salva i risultati in una nuova colonna chiamata `total_price_norm`





In [ ]:

# Prendo la colonna come array numpy
x = df_sales['total_price'].to_numpy(dtype=float)

# Calcolo min e max (ignorando eventuali NaN)
x_min = np.nanmin(x)
x_max = np.nanmax(x)

# Normalizzazione Min-Max: (x - min) / (max - min)
den = x_max - x_min
df_sales['total_price_norm'] = (x - x_min) / den if den != 0 else np.zeros_like(x)

# controllo
print(df_sales['total_price_norm'])


0      0.481390
1      0.460983
2      0.265458
3      0.632729
4      0.273384
         ...   
868    0.092588
869    0.101814
870    0.085785
871    0.203627
872    0.101814
Name: total_price_norm, Length: 873, dtype: float64


**Binning (Suddivisione in classi):**


*   Creare una nuova colonna `price_range` che categorizza il prezzo unitario dei prodotti
*   'Low' se prezzo < 100


*   'Medium' se 100 <= prezzo < 300
*   'High' se prezzo >= 300





In [ ]:

df_products['price_range'] = pd.cut(
    df_products['price'],
    bins=[-np.inf, 100, 300, np.inf],
    labels=['Low', 'Medium', 'High'],
    right=False  # intervalli: [a, b)
)

print(df_products['price_range'])

0    Medium
1    Medium
2    Medium
3    Medium
4    Medium
5    Medium
6      High
7       Low
8    Medium
9    Medium
Name: price_range, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']


# Parte 6: Preparazione per la Visualizzazione

Immagina di dover creare un grafico a barre (es. con Matplotlib o Seaborn).

**Aggregazione finale**



*   rea un DataFrame chiamato `df_chart` che contenga il numero di clienti unici (`customer_id)` per ogni brand.
*   Ordina questo DataFrame in modo decrescente


*   
*   Voce elenco






## **Parte 6: Preparazione per la Visualizzazione**

Immagina di dover creare un grafico a barre (es. con Matplotlib o Seaborn).

11. **Aggregazione Finale**:
    - Crea un DataFrame chiamato `df_chart` che contenga il numero di clienti unici (`customer_id`) per ogni `brand`.
    - Ordina questo DataFrame in modo decrescente.
    - Esporta l'output su un file CSV chiamato `report_vendite_brand.csv` senza indice.
          